In [41]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

In [42]:
with open("all_data.pkl", 'rb') as picklefile: 
    merge_housing = pickle.load(picklefile)
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy,Date,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting
0,Studio,2011,03,1097.5,2011,03,382800.0,37,33,24,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-03-01,NaN,NaN,NaN
1,Studio,2011,04,1095.0,2011,04,373500.0,33,22,16,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-04-01,NaN,NaN,NaN
2,Studio,2011,05,1025.0,2011,05,368400.0,33,25,21,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-05-01,NaN,NaN,NaN
3,Studio,2011,06,995.0,2011,06,359300.0,27,23,19,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-06-01,NaN,NaN,NaN
4,Studio,2011,07,1000.0,2011,07,365300.0,36,25,22,...,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369,2011-07-01,NaN,NaN,NaN


In [43]:

merge_housing['target_Apartment_rent'] = merge_housing['Apartment_rent']

merge_housing = merge_housing.dropna()
merge_housing.Year = merge_housing.Year.astype(int)
merge_housing.Month = merge_housing.Month.astype(int)
merge_housing.Search_Count_SeattleRent = merge_housing.Search_Count_SeattleRent.astype(int)
merge_housing.Search_Count_SeattleHousing = merge_housing.Search_Count_SeattleHousing.astype(int)
merge_housing.Search_Count_seattleApartment = merge_housing.Search_Count_seattleApartment.astype(int)
merge_housing['Avg Sale To List Mom'] = merge_housing['Avg Sale To List Mom'].str.replace('%','').astype(float)
merge_housing['Avg Sale To List Yoy'] = merge_housing['Avg Sale To List Yoy'].str.replace('%','').astype(float)
merge_housing['New Listings Mom'] = merge_housing['New Listings Mom'].str.replace('%','').astype(float)
merge_housing['New Listings Yoy'] = merge_housing['New Listings Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Mom'] = merge_housing['Median Sale Price Mom'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Yoy'] = merge_housing['Median Sale Price Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price'] = merge_housing['Median Sale Price'].str.replace('$','').str.replace('K','000').astype(float)
merge_housing['Median List Price'] = merge_housing['Median List Price'].str.replace(',','').astype(float)
merge_housing['New Listings'] = merge_housing['New Listings'].str.replace(',','').astype(float)


merge_housing['Homes Sold'] = merge_housing['Homes Sold'].str.replace(',','').astype(int)
merge_housing['Homes Sold Mom'] = merge_housing['Homes Sold Mom'].str.replace('%','').astype(float)
merge_housing['Homes Sold Yoy'] = merge_housing['Homes Sold Yoy'].str.replace('%','').astype(float)

merge_housing['Inventory'] = merge_housing['Inventory'].str.replace(',','').astype(int)
merge_housing['Inventory Mom'] = merge_housing['Inventory Mom'].str.replace('%','').astype(float)
merge_housing['Inventory Yoy'] = merge_housing['Inventory Yoy'].str.replace('%','').astype(float)
transfer_column = {k:k.replace(' ','') for k in merge_housing.iloc[:,6:-1].columns}


merge_housing = merge_housing.rename(columns=transfer_column)



In [44]:
merge_housing['count_news_seattle+shooting'] = merge_housing['count_news_seattle+shooting'].astype(int)
merge_housing['count_news_seattle+murder'] = merge_housing['count_news_seattle+murder'].astype(int)
merge_housing['count_news_seattle+dead+shooting'] = merge_housing['count_news_seattle+dead+shooting'].astype(int)

In [45]:
merge_housing.dtypes

BedSize                                     object
Year                                         int64
Month                                        int64
Apartment_rent                             float64
Shift_Year                                  object
Shift_Month                                 object
Zillow_Housing_price                       float64
Search_Count_SeattleRent                     int64
Search_Count_SeattleHousing                  int64
Search_Count_seattleApartment                int64
AvgSaleToList                              float64
AvgSaleToListMom                           float64
AvgSaleToListYoy                           float64
HomesSold                                    int64
HomesSoldMom                               float64
HomesSoldYoy                               float64
Inventory                                    int64
InventoryMom                               float64
InventoryYoy                               float64
MedianDom                      

In [46]:
# Save Data to Locale

In [47]:
with open('all_clean_data.pkl', 'wb') as picklefile:
    pickle.dump(merge_housing, picklefile)

In [48]:
with open("all_clean_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,PriceDropsMom,PriceDropsYoy,SoldAboveList,SoldAboveListMom,SoldAboveListYoy,Date,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting,target_Apartment_rent
28,Studio,2013,7,1188.0,2013,07,430600.0,40,25,22,...,0.039606,0.037088,0.277174,0.021826,0.032856,2013-07-01,8170,10900,5460,1188.0
29,Studio,2013,8,1199.0,2013,08,435300.0,37,23,24,...,-0.012473,0.044360,0.331746,0.054572,-0.007477,2013-08-01,9740,13900,6380,1199.0
30,Studio,2013,9,1231.0,2013,09,430100.0,42,27,27,...,0.024914,0.017092,0.404790,0.073044,0.041376,2013-09-01,4420,7230,3030,1231.0
31,Studio,2013,10,1180.0,2013,10,424600.0,40,27,26,...,0.004972,0.025687,0.453044,0.048254,0.043381,2013-10-01,10400,14800,6720,1180.0
32,Studio,2013,11,1205.0,2013,11,424900.0,46,27,28,...,0.010006,0.012604,0.439771,-0.013274,-0.013229,2013-11-01,10500,15900,7070,1205.0


In [49]:
my_old_data.dtypes

BedSize                                     object
Year                                         int64
Month                                        int64
Apartment_rent                             float64
Shift_Year                                  object
Shift_Month                                 object
Zillow_Housing_price                       float64
Search_Count_SeattleRent                     int64
Search_Count_SeattleHousing                  int64
Search_Count_seattleApartment                int64
AvgSaleToList                              float64
AvgSaleToListMom                           float64
AvgSaleToListYoy                           float64
HomesSold                                    int64
HomesSoldMom                               float64
HomesSoldYoy                               float64
Inventory                                    int64
InventoryMom                               float64
InventoryYoy                               float64
MedianDom                      